In [108]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

## Load the data into a DataFrame
djs = pd.read_csv('assets/Iowa_Liquor_sales_sample_10pct.csv', encoding='latin-1')
df = pd.read_csv('assets/iowa_15.csv', encoding='latin-1')

In [109]:
djs.dtypes

Date                      object
Store Number               int64
City                      object
Zip Code                  object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number              int64
Item Number                int64
Item Description          object
Bottle Volume (ml)         int64
State Bottle Cost         object
State Bottle Retail       object
Bottles Sold               int64
Sale (Dollars)            object
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object

In [112]:
#Converts State Bottle Cost Series to Float
djs['State Bottle Cost'] = (djs['State Bottle Cost'].replace( '[\$,)]','', regex=True )\
                            .replace( '[(]','-',   regex=True ).astype(float))

#Converts State Bottle Retail Series to Float
djs['State Bottle Retail'] = (djs['State Bottle Retail'].replace( '[\$,)]','', regex=True )\
                              .replace( '[(]','-',   regex=True ).astype(float))

#Converts Sale Dollars Series to Float
djs['Sale (Dollars)'] = (djs['Sale (Dollars)'].replace( '[\$,)]','', regex=True )\
                         .replace( '[(]','-',   regex=True ).astype(float))

In [43]:
type(djs['Sale (Dollars)'][5])

unicode

In [44]:
djs.columns

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')

In [118]:
print djs['City'].nunique()
print df['NAME'].nunique()

df['NAME'] = (df['NAME'].replace( '[\$,)]','', regex=True )\
                            .replace( '[(]','-',   regex=True ).astype(float))

hanks = pd.pivot_table(df, index=['NAME'], values=['POPESTIMATE2015'])
hanks

385
1210


,POPESTIMATE2015
NAME,
Ackley city,1560.0
Ackley city (pt.),780.0
Ackworth city,86.0
Adair County,7228.0
Adair city,728.0
Adair city (pt.),364.0
Adams County,3796.0
Adel city,4245.0
Afton city,829.0


In [79]:
i=0
for x in djs['County Number']:
    if x > 0:
        i+=1
print "Nulls:", 270954 - i

Nulls: 1076


In [80]:
t_hanks = pd.pivot_table(df, index=['County'], values=['County Number'])
t_hanks.head()

,County Number
County,
Adair,1.0
Adams,2.0
Allamakee,3.0
Appanoose,4.0
Audubon,5.0


In [64]:
djs['County'].unique()

array([u'Bremer', u'Scott', u'Black Hawk', u'Story', u'Wright', u'Linn',
       u'Wapello', u'Cerro Gordo', u'Polk', u'Benton', u'Mahaska', u'Lee',
       u'Johnson', u'Jasper', u'Tama', u'Sioux', u'Dickinson',
       u'Pottawattamie', u'Warren', u'Webster', u'Boone', u'Jackson',
       u'Carroll', u'Hamilton', u'Woodbury', u'Washington', u'Floyd',
       u'Delaware', u"O'Brien", u'Buena Vista', u'Grundy', u'Dubuque',
       u'Adair', u'Crawford', u'Henry', u'Guthrie', u'Winneshiek', u'Clay',
       u'Muscatine', u'Taylor', u'Harrison', u'Hardin', u'Kossuth',
       u'Dallas', u'Decatur', u'Clinton', u'Butler', u'Des Moines',
       u'Calhoun', u'Fayette', u'Jones', nan, u'Hancock', u'Iowa',
       u'Howard', u'Appanoose', u'Jefferson', u'Emmet', u'Marshall',
       u'Page', u'Cedar', u'Clayton', u'Sac', u'Monona', u'Allamakee',
       u'Winnebago', u'Poweshiek', u'Marion', u'Buchanan', u'Franklin',
       u'Cherokee', u'Humboldt', u'Greene', u'Keokuk', u'Plymouth',
       u'Shelby', u

In [48]:
djs['Sale (Dollars)'].head()

0     $81.00
1     $41.26
2    $453.36
3     $85.50
4    $129.60
Name: Sale (Dollars), dtype: object

In [115]:
df.columns

Index([u'SUMLEV', u'STATE', u'COUNTY', u'PLACE', u'COUSUB', u'CONCIT',
       u'PRIMGEO_FLAG', u'FUNCSTAT', u'NAME', u'STNAME', u'CENSUS2010POP',
       u'ESTIMATESBASE2010', u'POPESTIMATE2010', u'POPESTIMATE2011',
       u'POPESTIMATE2012', u'POPESTIMATE2013', u'POPESTIMATE2014',
       u'POPESTIMATE2015'],
      dtype='object')